## A gentle 10-minute primer to Ray AI Runitime (Ray AIR)

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

### Overview

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source and unified toolkit for building end-to-end simple and scalable ML applications. 

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key components:

 * Data processing ([Ray Data](https://docs.ray.io/en/latest/data/dataset.html))
 * Model Training ([Ray Train](https://docs.ray.io/en/latest/train/train.html))
 * Hyperparameter Tuning ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html))
 * Model Serving ([Ray Serve](https://docs.ray.io/en/latest/serve/index.html)).
 * Reinforcement Learning ([Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html))
 
 <img src = "images/ray-air.svg" width="60%" height="30%">
 
📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>
 
### Learning objectives:
  * How to use Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Use out-of-box Preprocessors
  * Load model from the best model checkpoint and use for batch inference
  * Deploy best checkpoint model and use for online inference

In [1]:
import logging, os, random, warnings
import ray
import pandas as pd

In [2]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc1
Dashboard:,http://127.0.0.1:8266


### End-to-end ML stages for a Ray AIR ML application

<img src="images/ray_air_pipeline.png" width="50%" height="25%">

### 1. Create Ray data from an S3 CSV datasource

In [4]:
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.79it/s]


### 2. Use out-of-the-box Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation. You don't have to explicitly call the preprocess before training or inference. Ray AIR toolkit automatically does that for you. 

We are going to scaler a few features like `mean radius` and `mean texture`.

In [5]:
from ray.data.preprocessors import StandardScaler

# Create a preprocessor to scale some columns
columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### 3a. Use AIR Trainers for supported ML frameworks
Use the Ray AIR trainer `XGBoostTrainer` with simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets for training and validation
 5. invoke `trainer.fit()` 
 
 Simple API that does a lot behind the scenes for you!

In [6]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=2,
        # Whether to use GPU acceleration.
        use_gpu=False),
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    # our train and validation dataset and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)

##### Fit the trainer

In [7]:
result = trainer.fit()
# print(result.metrics)

Trial name,status,loc,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_60c8f_00000,TERMINATED,127.0.0.1:53280,21,4.75553,0.0184957,0,0.0893879


(XGBoostTrainer pid=53280) 2022-08-14 16:04:00,664	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=53296) 2022-08-14 16:04:01,851	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=53296)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=53296)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=53296)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=53296)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=53296)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_60c8f_00000:
  date: 2022-08-14_16-04-03
  done: false
  experiment_id: f76ce683ab074b018ee8abe73597d867
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 53280
  time_since_restore: 4.218009948730469
  time_this_iter_s: 4.218009948730469
  time_total_s: 4.218009948730469
  timestamp: 1660518243
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 60c8f_00000
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.0026197433471679688
  


(XGBoostTrainer pid=53280) 2022-08-14 16:04:04,020	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.37 seconds (1.74 pure XGBoost training time).


Result for XGBoostTrainer_60c8f_00000:
  date: 2022-08-14_16-04-04
  done: true
  experiment_id: f76ce683ab074b018ee8abe73597d867
  experiment_tag: '0'
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 53280
  time_since_restore: 4.75553297996521
  time_this_iter_s: 0.4674220085144043
  time_total_s: 4.75553297996521
  timestamp: 1660518244
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: 60c8f_00000
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.0026197433471679688
  


### 3b. Use AIR Tuner for hyperparameter search

What if you want to do hyperparameter optimization during training and use the best config for the model? Well, you can then use Tuner and supply your training function, Trainer, as part of the argument, along 
with other Tuner configuration. 

Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [8]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"
our_mode="min"

In [9]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode=our_mode),
)
# Execute tuning.
result_grid = tuner.fit()

Trial name,status,loc,params/max_depth,iter,total time (s),train-logloss,train-error,valid-logloss
XGBoostTrainer_699ab_00000,TERMINATED,127.0.0.1:53313,4,21,3.86651,0.0183889,0,0.100561
XGBoostTrainer_699ab_00001,TERMINATED,127.0.0.1:53318,5,21,4.92101,0.0184163,0,0.105782
XGBoostTrainer_699ab_00002,TERMINATED,127.0.0.1:53319,5,21,4.96536,0.0184163,0,0.105782
XGBoostTrainer_699ab_00003,TERMINATED,127.0.0.1:53369,6,21,4.80091,0.0184957,0,0.0893879
XGBoostTrainer_699ab_00004,TERMINATED,127.0.0.1:53395,6,21,4.86592,0.0184957,0,0.0893879


(XGBoostTrainer pid=53313) 2022-08-14 16:04:14,378	INFO main.py:980 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(_RemoteRayXGBoostActor pid=53325) 2022-08-14 16:04:15,603	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.get_resource_ids` is a private attribute and access will be removed in a future Ray version.
(_RemoteRayXGBoostActor pid=53325)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/workers/default_worker.py", line 237, in <module>
(_RemoteRayXGBoostActor pid=53325)     ray._private.worker.global_worker.main_loop()
(_RemoteRayXGBoostActor pid=53325)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/worker.py", line 754, in main_loop
(_RemoteRayXGBoostActor pid=53325)     self.core_worker.run_task_loop()
(_RemoteRayXGBoostActor pid=53325)   File "/opt/miniconda3/envs/ray-summit-training/lib/python3.8/site-packages/ray/_private/func

Result for XGBoostTrainer_699ab_00000:
  date: 2022-08-14_16-04-17
  done: false
  experiment_id: 21664c2026f04418a0a640cd783a9dea
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 53313
  time_since_restore: 3.478624105453491
  time_this_iter_s: 3.478624105453491
  time_total_s: 3.478624105453491
  timestamp: 1660518257
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.4666400034373729
  training_iteration: 1
  trial_id: 699ab_00000
  valid-error: 0.0935672514619883
  valid-logloss: 0.4943798714213901
  warmup_time: 0.0026497840881347656
  


(XGBoostTrainer pid=53313) 2022-08-14 16:04:17,845	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 3.48 seconds (2.07 pure XGBoost training time).
(XGBoostTrainer pid=53318) 2022-08-14 16:04:18,063	INFO main.py:1025 -- [RayXGBoost] Starting XGBoost training.
(XGBoostTrainer pid=53319) 2022-08-14 16:04:18,067	INFO main.py:1025 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=53339) [16:04:18] task [xgboost.ray]:4952657536 got new rank 1
(_RemoteRayXGBoostActor pid=53338) [16:04:18] task [xgboost.ray]:5117968048 got new rank 0
(_RemoteRayXGBoostActor pid=53346) [16:04:18] task [xgboost.ray]:6096895568 got new rank 0
(_RemoteRayXGBoostActor pid=53347) [16:04:18] task [xgboost.ray]:4802330336 got new rank 1


Result for XGBoostTrainer_699ab_00000:
  date: 2022-08-14_16-04-18
  done: true
  experiment_id: 21664c2026f04418a0a640cd783a9dea
  experiment_tag: 0_max_depth=4
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 53313
  time_since_restore: 3.8665130138397217
  time_this_iter_s: 0.32891392707824707
  time_total_s: 3.8665130138397217
  timestamp: 1660518258
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01838890815109258
  training_iteration: 21
  trial_id: 699ab_00000
  valid-error: 0.04678362573099415
  valid-logloss: 0.10056091369026238
  warmup_time: 0.0026497840881347656
  
Result for XGBoostTrainer_699ab_00002:
  date: 2022-08-14_16-04-19
  done: false
  experiment_id: a80d0fe1aa414c9ab3f32529fd83a2a9
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 53319
  time_since_restore: 4.254423141479492
  time_this_iter_s: 4.254423141479492
  time_total_s: 4.254423141479492
  timestamp:

...

Result for XGBoostTrainer_699ab_00003:
  date: 2022-08-14_16-04-24
  done: true
  experiment_id: a500263f21df4355a727781dee7e23de
  experiment_tag: 3_max_depth=6
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 53369
  time_since_restore: 4.800912141799927
  time_this_iter_s: 0.5003941059112549
  time_total_s: 4.800912141799927
  timestamp: 1660518264
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: 699ab_00003
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.0027399063110351562
  


(XGBoostTrainer pid=53395) 2022-08-14 16:04:25,617	INFO main.py:1025 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=53402) [16:04:25] task [xgboost.ray]:5024153072 got new rank 1
(_RemoteRayXGBoostActor pid=53401) [16:04:25] task [xgboost.ray]:5144477120 got new rank 0


Result for XGBoostTrainer_699ab_00004:
  date: 2022-08-14_16-04-27
  done: false
  experiment_id: 00b589f4ece145169c6d3f9b088eee20
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 53395
  time_since_restore: 4.055120944976807
  time_this_iter_s: 4.055120944976807
  time_total_s: 4.055120944976807
  timestamp: 1660518267
  timesteps_since_restore: 0
  train-error: 0.02261306532663317
  train-logloss: 0.464117960489575
  training_iteration: 1
  trial_id: 699ab_00004
  valid-error: 0.11695906432748537
  valid-logloss: 0.5025240946234318
  warmup_time: 0.002563953399658203
  


(XGBoostTrainer pid=53395) 2022-08-14 16:04:27,395	INFO main.py:1516 -- [RayXGBoost] Finished XGBoost training on training data with total N=398 in 4.09 seconds (1.77 pure XGBoost training time).


Result for XGBoostTrainer_699ab_00004:
  date: 2022-08-14_16-04-28
  done: true
  experiment_id: 00b589f4ece145169c6d3f9b088eee20
  experiment_tag: 4_max_depth=6
  hostname: Juless-MacBook-Pro-16
  iterations_since_restore: 21
  node_ip: 127.0.0.1
  pid: 53395
  time_since_restore: 4.865922927856445
  time_this_iter_s: 0.7292649745941162
  time_total_s: 4.865922927856445
  timestamp: 1660518268
  timesteps_since_restore: 0
  train-error: 0.0
  train-logloss: 0.01849572773292735
  training_iteration: 21
  trial_id: 699ab_00004
  valid-error: 0.04093567251461988
  valid-logloss: 0.08938791319913073
  warmup_time: 0.002563953399658203
  


In [10]:
# Fetch the best result with its best hyperparameter config 
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

Best Result: Result(metrics={'train-logloss': 0.01838890815109258, 'train-error': 0.0, 'valid-logloss': 0.10056091369026238, 'valid-error': 0.04678362573099415, 'done': True, 'trial_id': '699ab_00000', 'experiment_tag': '0_max_depth=4'}, error=None, log_dir=PosixPath('/Users/jules/ray_results/XGBoostTrainer_2022-08-14_16-04-12/XGBoostTrainer_699ab_00000_0_max_depth=4_2022-08-14_16-04-13'))


### Ray AIR Checkpoints

The AIR trainers, tuners, and custom pretrained model generate Checkpoints. An AIR Checkpoint is a format for models that are used across different components of the Ray AI Runtime. This common format allows easy interoperability among AIR components and seamless integration with external supported machine learning frameworks. Read more
about [Checkpoints]().

<img src="images/checkpoints.jpeg" height="25%" and width="50%"> 

### 4. Use AIR `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

In [11]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

Map Progress (1 actors 0 pending): 100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.30it/s]

PREDICTED PROBABILITIES
{'predictions': 0.9966920614242554}
{'predictions': 0.9931760430335999}
{'predictions': 0.0034648359287530184}
{'predictions': 0.9966920614242554}
{'predictions': 0.9965646862983704}
{'predictions': 0.9956005811691284}
{'predictions': 0.9950228929519653}
{'predictions': 0.9943311214447021}
{'predictions': 0.4793323874473572}
{'predictions': 0.9818810820579529}
{'predictions': 0.0034648359287530184}
{'predictions': 0.996193528175354}
{'predictions': 0.9557499885559082}
{'predictions': 0.993036687374115}
{'predictions': 0.9940920472145081}
{'predictions': 0.22775237262248993}
{'predictions': 0.4834454357624054}
{'predictions': 0.9949895739555359}
{'predictions': 0.9798774123191833}
{'predictions': 0.0034648359287530184}


### 5. Use `PredictorDeployment` for online inference

Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [12]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import json_request

# Ray AIR supports few convenient adapters such as http_request to process request.
async def adapter(request: Request):
    content = await request.json()
    print(content)
    return pd.DataFrame.from_dict(content)


serve.start(detached=True)
deployment = PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair")

deployment.deploy(
    XGBoostPredictor, best_result.checkpoint, batching_params=False, http_adapter=adapter
)

print(deployment.url)

(ServeController pid=53510) INFO 2022-08-14 16:04:47,583 controller 53510 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-818c29dabf6112271f21e1a39a659bac89b22d485dc59d39014e410a' on node '818c29dabf6112271f21e1a39a659bac89b22d485dc59d39014e410a' listening on '127.0.0.1:8000'
(ServeController pid=53510) INFO 2022-08-14 16:04:48,201 controller 53510 deployment_state.py:1232 - Adding 2 replicas to deployment 'XGBoostService'.
(HTTPProxyActor pid=53513) INFO:     Started server process [53513]


http://127.0.0.1:8000/rayair


After deploying the service, you can send requests to it.

In [13]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post(deployment.url, json=[sample_input]).json()
print(output)

[{'predictions': 0.9966920614242554}]
(ServeReplica:XGBoostService pid=53515) [{'mean radius': 11.06, 'mean texture': 14.83, 'mean perimeter': 70.31, 'mean area': 378.2, 'mean smoothness': 0.07741, 'mean compactness': 0.04768, 'mean concavity': 0.02712, 'mean concave points': 0.007246, 'mean symmetry': 0.1535, 'mean fractal dimension': 0.06214, 'radius error': 0.1855, 'texture error': 0.6881, 'perimeter error': 1.263, 'area error': 12.98, 'smoothness error': 0.004259, 'compactness error': 0.01469, 'concavity error': 0.0194, 'concave points error': 0.004168, 'symmetry error': 0.01191, 'fractal dimension error': 0.003537, 'worst radius': 12.68, 'worst texture': 20.35, 'worst perimeter': 80.79, 'worst area': 496.7, 'worst smoothness': 0.112, 'worst compactness': 0.1879, 'worst concavity': 0.2079, 'worst concave points': 0.05556, 'worst symmetry': 0.259, 'worst fractal dimension': 0.09158}]


(HTTPProxyActor pid=53513) INFO 2022-08-14 16:04:53,884 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 307 3.3ms
(HTTPProxyActor pid=53513) INFO 2022-08-14 16:04:53,895 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 200 9.3ms
(ServeReplica:XGBoostService pid=53517) INFO 2022-08-14 16:04:53,884 XGBoostService XGBoostService#gOfHfH replica.py:482 - HANDLE __call__ OK 0.2ms
(ServeReplica:XGBoostService pid=53515) INFO 2022-08-14 16:04:53,894 XGBoostService XGBoostService#BaynLs replica.py:482 - HANDLE __call__ OK 6.2ms


In [14]:
ray.shutdown()

### Homework

1. Have a go at Ray AIR examples in the documentation.

 📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

Done! 🍻
 